In [1]:
# import os
# import cv2
# import numpy as np
# from keras.models import model_from_json
# from keras.preprocessing import image

# #load model
# model = model_from_json(open("fer.json", "r").read())
# #load weights
# model.load_weights('fer.h5')


# face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


# cap=cv2.VideoCapture(0)

# while True:
#     ret,test_img=cap.read()# captures frame and returns boolean value and captured image
#     if not ret:
#         continue
#     gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

#     faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


#     for (x,y,w,h) in faces_detected:
#         cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
#         roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
#         roi_gray=cv2.resize(roi_gray,(48,48))
#         img_pixels = image.img_to_array(roi_gray)
#         img_pixels = np.expand_dims(img_pixels, axis = 0)
#         img_pixels /= 255

#         predictions = model.predict(img_pixels)

#         #find max indexed array
#         max_index = np.argmax(predictions[0])

#         emotions = ('angry', 'disgust', 'happy', 'sad', 'surprise', 'neutral', 'fear')
#         predicted_emotion = emotions[max_index]

#         cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

#     resized_img = cv2.resize(test_img, (1000, 700))
#     cv2.imshow('Facial emotion analysis ',resized_img)



#     if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed
#         break

# cap.release()
# cv2.destroyAllWindows

In [1]:

import logging
import sys
import numpy as np
from keras.models import Model
from keras.layers import Input, Activation, add, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

sys.setrecursionlimit(2 ** 20)
np.random.seed(2 ** 10)


class WideResNet:
    def __init__(self, image_size, depth=16, k=8):
        self._depth = depth
        self._k = k
        self._dropout_probability = 0
        self._weight_decay = 0.0005
        self._use_bias = False
        self._weight_init = "he_normal"

        if K.common.image_dim_ordering() == "th":
            logging.debug("image_dim_ordering = 'th'")
            self._channel_axis = 1
            self._input_shape = (3, image_size, image_size)
        else:
            logging.debug("image_dim_ordering = 'tf'")
            self._channel_axis = -1
            self._input_shape = (image_size, image_size, 3)

    # Wide residual network http://arxiv.org/abs/1605.07146
    def _wide_basic(self, n_input_plane, n_output_plane, stride):
        def f(net):
            # format of conv_params:
            #               [ [kernel_size=("kernel width", "kernel height"),
            #               strides="(stride_vertical,stride_horizontal)",
            #               padding="same" or "valid"] ]
            # B(3,3): orignal <<basic>> block
            conv_params = [[3, 3, stride, "same"],
                           [3, 3, (1, 1), "same"]]

            n_bottleneck_plane = n_output_plane

            # Residual block
            for i, v in enumerate(conv_params):
                if i == 0:
                    if n_input_plane != n_output_plane:
                        net = BatchNormalization(axis=self._channel_axis)(net)
                        net = Activation("relu")(net)
                        convs = net
                    else:
                        convs = BatchNormalization(axis=self._channel_axis)(net)
                        convs = Activation("relu")(convs)

                    convs = Conv2D(n_bottleneck_plane, kernel_size=(v[0], v[1]),
                                          strides=v[2],
                                          padding=v[3],
                                          kernel_initializer=self._weight_init,
                                          kernel_regularizer=l2(self._weight_decay),
                                          use_bias=self._use_bias)(convs)
                else:
                    convs = BatchNormalization(axis=self._channel_axis)(convs)
                    convs = Activation("relu")(convs)
                    if self._dropout_probability > 0:
                        convs = Dropout(self._dropout_probability)(convs)
                    convs = Conv2D(n_bottleneck_plane, kernel_size=(v[0], v[1]),
                                          strides=v[2],
                                          padding=v[3],
                                          kernel_initializer=self._weight_init,
                                          kernel_regularizer=l2(self._weight_decay),
                                          use_bias=self._use_bias)(convs)

            # Shortcut Connection: identity function or 1x1 convolutional
            #  (depends on difference between input & output shape - this
            #   corresponds to whether we are using the first block in each
            #   group; see _layer() ).
            if n_input_plane != n_output_plane:
                shortcut = Conv2D(n_output_plane, kernel_size=(1, 1),
                                         strides=stride,
                                         padding="same",
                                         kernel_initializer=self._weight_init,
                                         kernel_regularizer=l2(self._weight_decay),
                                         use_bias=self._use_bias)(net)
            else:
                shortcut = net

            return add([convs, shortcut])

        return f


    # "Stacking Residual Units on the same stage"
    def _layer(self, block, n_input_plane, n_output_plane, count, stride):
        def f(net):
            net = block(n_input_plane, n_output_plane, stride)(net)
            for i in range(2, int(count + 1)):
                net = block(n_output_plane, n_output_plane, stride=(1, 1))(net)
            return net

        return f

#    def create_model(self):
    def __call__(self):
        logging.debug("Creating model...")

        assert ((self._depth - 4) % 6 == 0)
        n = (self._depth - 4) / 6

        inputs = Input(shape=self._input_shape)

        n_stages = [16, 16 * self._k, 32 * self._k, 64 * self._k]

        conv1 = Conv2D(filters=n_stages[0], kernel_size=(3, 3),
                              strides=(1, 1),
                              padding="same",
                              kernel_initializer=self._weight_init,
                              kernel_regularizer=l2(self._weight_decay),
                              use_bias=self._use_bias)(inputs)  # "One conv at the beginning (spatial size: 32x32)"

        # Add wide residual blocks
        block_fn = self._wide_basic
        conv2 = self._layer(block_fn, n_input_plane=n_stages[0], n_output_plane=n_stages[1], count=n, stride=(1, 1))(conv1)
        conv3 = self._layer(block_fn, n_input_plane=n_stages[1], n_output_plane=n_stages[2], count=n, stride=(2, 2))(conv2)
        conv4 = self._layer(block_fn, n_input_plane=n_stages[2], n_output_plane=n_stages[3], count=n, stride=(2, 2))(conv3)
        batch_norm = BatchNormalization(axis=self._channel_axis)(conv4)
        relu = Activation("relu")(batch_norm)

        # Classifier block
        pool = AveragePooling2D(pool_size=(8, 8), strides=(1, 1), padding="same")(relu)
        flatten = Flatten()(pool)
        predictions_g = Dense(units=2, kernel_initializer=self._weight_init, use_bias=self._use_bias,
                              kernel_regularizer=l2(self._weight_decay), activation="softmax")(flatten)
        predictions_a = Dense(units=101, kernel_initializer=self._weight_init, use_bias=self._use_bias,
                              kernel_regularizer=l2(self._weight_decay), activation="softmax")(flatten)

        model = Model(inputs=inputs, outputs=[predictions_g, predictions_a])

        return model



Using TensorFlow backend.
C:\Users\Asad Shah\anaconda3\envs\ai2_course\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Asad Shah\anaconda3\envs\ai2_course\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Asad Shah\anaconda3\envs\ai2_course\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Asad Shah\a

In [3]:
"""
Face detection
"""
import cv2
import os
from time import sleep
import numpy as np
import argparse
from keras.utils.data_utils import get_file
from keras.models import model_from_json
from keras.preprocessing import image


#load model
model = model_from_json(open("newModel2.json", "r").read())
#load weights
model.load_weights('newModel2.h5')


face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#This module captures images via webcam and performs face recognition
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('trainingData.yml')#Load saved training data

# name = {0: "Unknown", 1:"Priyanka",2:"Kangana"}
name = {0:"Asad",1:"Leo Varadkar",2:"Priyanka",3:"Kangana"}

class FaceCV(object):
    
  
    """
    Singleton class for face recongnition task
    """
    CASE_PATH = "C:/Users/Asad Shah/Documents/final/final/pretrained_models/haarcascade_frontalface_alt.xml"
    WRN_WEIGHTS_PATH = "C:/Users/Asad Shah/Documents/final/final/weights.18-4.06.hdf5"


    def __new__(cls, weight_file=None, depth=16, width=8, face_size=64):
        if not hasattr(cls, 'instance'):
            cls.instance = super(FaceCV, cls).__new__(cls)
        return cls.instance

    def __init__(self, depth=16, width=8, face_size=64):
        self.face_size = face_size
        self.model = WideResNet(face_size, depth=depth, k=width)()
        model_dir = os.path.join(os.getcwd(), "pretrained_models").replace("//", "\\")
#         fpath = get_file('weights.18-4.06.hdf5',
#                          self.WRN_WEIGHTS_PATH,
#                          cache_subdir=model_dir)
        self.model.load_weights("C:/Users/Asad Shah/Documents/final/final/weights.18-4.06.hdf5")

    @classmethod
    def draw_label(cls, image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
                   font_scale=1, thickness=2):
        size = cv2.getTextSize(label, font, font_scale, thickness)[0]
        x, y = point
        cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
        cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness)

    def crop_face(self, imgarray, section, margin=40, size=64):
        """
        :param imgarray: full image
        :param section: face detected area (x, y, w, h)
        :param margin: add some margin to the face detected area to include a full head
        :param size: the result image resolution with be (size x size)
        :return: resized image in numpy array with shape (size x size x 3)
        """
        img_h, img_w, _ = imgarray.shape
        if section is None:
            section = [0, 0, img_w, img_h]
        (x, y, w, h) = section
        margin = int(min(w,h) * margin / 100)
        x_a = x - margin
        y_a = y - margin
        x_b = x + w + margin
        y_b = y + h + margin
        if x_a < 0:
            x_b = min(x_b - x_a, img_w-1)
            x_a = 0
        if y_a < 0:
            y_b = min(y_b - y_a, img_h-1)
            y_a = 0
        if x_b > img_w:
            x_a = max(x_a - (x_b - img_w), 0)
            x_b = img_w
        if y_b > img_h:
            y_a = max(y_a - (y_b - img_h), 0)
            y_b = img_h
        cropped = imgarray[y_a: y_b, x_a: x_b]
        resized_img = cv2.resize(cropped, (size, size), interpolation=cv2.INTER_AREA)
        resized_img = np.array(resized_img)
        return resized_img, (x_a, y_a, x_b - x_a, y_b - y_a)

    def detect_face(self):
            
   
   
        face_cascade = cv2.CascadeClassifier(self.CASE_PATH)

        # 0 means the default video capture device in OS
        video_capture = cv2.VideoCapture(0)
        


        # infinite loop, break by key ESC
        while True:
            if not video_capture.isOpened():
                sleep(5)
            # Capture frame-by-frame
            ret, frame = video_capture.read()
            
            if not ret:
                continue
            gray_img= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)

            predicted_emotion = ''

            for (x,y,w,h) in faces_detected:
                face = (x,y,w,h)
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),thickness=7)
                roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
                roi_gray=cv2.resize(roi_gray,(48,48))
                img_pixels = image.img_to_array(roi_gray)
                img_pixels = np.expand_dims(img_pixels, axis = 0)
                img_pixels /= 255
                            
                            
                predictions = model.predict(img_pixels)

                #find max indexed array
                max_index = np.argmax(predictions[0])
                print(max_index)
                emotions = ('angry', 'disgust', 'happy', 'fear', 'sad', 'surprise', 'neutral')
                predicted_emotion = emotions[max_index]

                predicted_name=''
                
                label,confidence=face_recognizer.predict(gray_img[y:y+w,x:x+h])#predicting the label of given image
                print("confidence:",confidence)
                print("label:",name[label])
                if confidence < 55:#If confidence less than 55 then don't print predicted face text on screen
                    predicted_name=name[label]
                else:
                    predicted_name='Unknown'
                            
                            
#                 cv2.putText(frame, predicted_emotion, (int(x+w), int(y+h)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

#             frame = cv2.resize(frame, (1000, 700))
            
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.2,
                minNeighbors=10,
                minSize=(self.face_size, self.face_size)
            )
            if faces is not ():
                
                # placeholder for cropped faces
                face_imgs = np.empty((len(faces), self.face_size, self.face_size, 3))
                for i, face in enumerate(faces):
                    face_img, cropped = self.crop_face(frame, face, margin=40, size=self.face_size)
                    (x, y, w, h) = cropped
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 200, 0), 2)
                    face_imgs[i,:,:,:] = face_img
               
                if len(face_imgs) > 0:
                    # predict ages and genders of the detected faces
                    results = self.model.predict(face_imgs)
                    predicted_genders = results[0]
                    ages = np.arange(0, 101).reshape(101, 1)
                    predicted_ages = results[1].dot(ages).flatten()
                   
                # draw results
                for i, face in enumerate(faces):
                    label = "{}, {}, {}, {}".format(predicted_name, int(predicted_ages[i]),
                                            "F" if predicted_genders[i][0] > 0.5 else "M",
                                               predicted_emotion)
                    
                    self.draw_label(frame, (face[0], face[1]), label)
            else:
                # print('No faces')
                pass

            cv2.imshow('Keras Faces', frame)
            if cv2.waitKey(5) == 27:  # ESC key press
                break
        # When everything is done, release the capture
        video_capture.release()
        cv2.destroyAllWindows()


def main():
    depth = 16
    width = 8

    face = FaceCV(depth=depth, width=width)

    face.detect_face()

main()




2
confidence: 142.21114901698937
label: Priyanka
2
confidence: 106.06360652843134
label: Priyanka
2
confidence: 85.67586890814025
label: Priyanka
2
confidence: 85.92949387242444
label: Priyanka
2
confidence: 92.77189724301277
label: Priyanka
2
confidence: 97.59719705524375
label: Kangana
2
confidence: 94.68627632718594
label: Kangana
2
confidence: 90.00736815578401
label: Priyanka
2
confidence: 88.1202587852345
label: Priyanka
2
confidence: 90.02993522371618
label: Priyanka
2
confidence: 98.13653089839539
label: Kangana
2
confidence: 94.45111718112744
label: Priyanka
2
confidence: 98.35598699485723
label: Priyanka
2
confidence: 105.71097789534569
label: Priyanka
2
confidence: 98.89909799442242
label: Priyanka
2
confidence: 109.01267536409817
label: Kangana
6
confidence: 99.30181394913994
label: Kangana
2
confidence: 96.33077736125175
label: Priyanka
2
confidence: 90.50541558741104
label: Priyanka
6
confidence: 89.67356071922947
label: Priyanka
2
confidence: 92.17990507574812
label: Pri

confidence: 144.67274557803466
label: Priyanka
3
confidence: 122.65086393322603
label: Kangana
6
confidence: 126.45604554607185
label: Priyanka
2
confidence: 116.31587795684403
label: Priyanka
3
confidence: 120.74728226984489
label: Priyanka
2
confidence: 119.59588812740427
label: Priyanka
3
confidence: 116.17806091351595
label: Kangana
2
confidence: 119.57780794758318
label: Priyanka
3
confidence: 117.41829148211623
label: Kangana
2
confidence: 120.91961899499829
label: Priyanka
3
confidence: 115.02896610400738
label: Kangana
2
confidence: 156.7435245749372
label: Priyanka
2
confidence: 150.23838817669022
label: Priyanka
2
confidence: 151.9165608130619
label: Priyanka
2
confidence: 147.92229379923
label: Priyanka
2
confidence: 142.86426092060006
label: Priyanka
2
confidence: 125.64376632970377
label: Priyanka
2
confidence: 123.18327153579168
label: Priyanka
3
confidence: 126.15816394547957
label: Kangana
2
confidence: 135.35037181893605
label: Priyanka
2
confidence: 135.38442534570598

confidence: 46.242268005780524
label: Asad
2
confidence: 46.15677755425423
label: Asad
2
confidence: 45.97770582664254
label: Asad
2
confidence: 44.071279420460314
label: Asad
2
confidence: 44.19013680900413
label: Asad
2
confidence: 44.43659154317798
label: Asad
2
confidence: 47.73586391082442
label: Asad
2
confidence: 46.35230806032648
label: Asad
2
confidence: 48.99391494528862
label: Asad
2
confidence: 46.95636289697973
label: Asad
2
confidence: 42.96929257574531
label: Asad
2
confidence: 41.16184245438555
label: Asad
2
confidence: 40.95843721315964
label: Asad
2
confidence: 152.7376487245795
label: Asad
2
confidence: 63.11556674697042
label: Leo Varadkar
2
confidence: 65.62749018412987
label: Leo Varadkar
3
confidence: 67.63064198759338
label: Leo Varadkar
6
confidence: 69.17573530060496
label: Leo Varadkar
2
confidence: 68.58822504186827
label: Leo Varadkar
6
confidence: 69.24171197005492
label: Leo Varadkar
6
confidence: 76.92670082614467
label: Leo Varadkar
3
confidence: 84.752

2
confidence: 112.93784846231229
label: Asad
2
confidence: 118.24053470767385
label: Kangana
3
confidence: 110.95778966846325
label: Kangana
6
confidence: 57.71484823084651
label: Asad
2
confidence: 53.268246146675146
label: Asad
6
confidence: 52.618545637351474
label: Asad
6
confidence: 45.12587296425473
label: Asad
6
confidence: 44.76916083645652
label: Asad
2
confidence: 47.34519583794986
label: Asad
2
confidence: 45.86379355781368
label: Asad
6
confidence: 42.49084462487587
label: Asad
6
confidence: 41.65161289859058
label: Asad
6
confidence: 43.059014288598924
label: Asad
6
confidence: 43.34508417811731
label: Asad
2
confidence: 44.13845823549253
label: Asad
6
confidence: 43.483652643598774
label: Asad
6
confidence: 40.024684512728044
label: Asad
6
confidence: 39.22317105481257
label: Asad
6
confidence: 39.146688560721046
label: Asad
6
confidence: 39.39208442259713
label: Asad
6
confidence: 40.947351318916176
label: Asad
6
confidence: 38.76783438052052
label: Asad
6
confidence: 38